## EditDistance

In [1]:
def editDistRecursive(x, y):
    # This implementation is very slow
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    else:
        distHor = editDistRecursive(x[:-1], y) + 1
        distVer = editDistRecursive(x, y[:-1]) + 1
        if x[-1] == y[-1]:
            distDiag = editDistRecursive(x[:-1], y[:-1])
        else:
            distDiag = editDistRecursive(x[:-1], y[:-1]) + 1
        return min(distHor, distVer, distDiag)

In [50]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [48]:
%%time
x = 'shake spea'
y = 'Shakespear'
editDistRecursive(x, y)

3

In [52]:
%%time
x = 'shake spea'
y = 'Shakespear'
editDistance(x, y)

CPU times: user 167 µs, sys: 3 µs, total: 170 µs
Wall time: 174 µs


## Global Alignment

In [5]:
alphabet = ['A', 'C', 'G', 'T']
score = [[0, 4, 2, 4, 8],
         [4, 0, 4, 2, 8],
         [2, 4, 0, 4, 8],
         [4, 2, 4, 0, 8],
         [8, 8, 8, 8, 8]]

In [6]:
# converts from character to its offset in list alphabet
alphabet.index('A')

0

In [7]:
alphabet.index('G')

2

In [8]:
# penalty associated with A (from X) mismatching with T (from Y)
score[alphabet.index('A')][alphabet.index('T')]

4

In [9]:
# penalty associated with C (from X) being deleted in Y
score[alphabet.index('C')][-1]

8

In [10]:
def globalAlignment(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0] * (len(y)+1))
        
    # Initialize first column
    for i in range(1, len(x)+1):
        D[i][0] = D[i-1][0] + score[alphabet.index(x[i-1])][-1] # -8

    # Initialize first row
    for j in range(1,len(y)+1):
        D[0][j] = D[0][j-1] + score[-1][alphabet.index(y[j-1])] # -8
        
    # Fill rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + score[-1][alphabet.index(y[j-1])]
            distVer = D[i-1][j] + score[alphabet.index(x[i-1])][-1]
            distDiag = D[i-1][j-1] + score[alphabet.index(x[i-1])][alphabet.index(y[j-1])]
            D[i][j] = min(distHor, distVer, distDiag)
    
    return D[-1][-1]  # return value in bottom right corner

In [11]:
x = 'TATGTCATGC'
y = 'TATGGCAGC'
print(globalAlignment(x,y))

12


## Finding Overlaps

In [41]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [42]:
overlap('TTACGT', 'CGTGTGC')

3

In [43]:
overlap('TTACGT', 'GTGTGC')

0

## FInding all overlaps

In [ ]:
#visto en la clase anterior
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [44]:
from itertools import permutations

list(permutations([1,2,3], 2))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

In [45]:
def naive_overlap_map(reads, k):
    olaps = {}
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0:
            olaps[(a, b)] = olen
    return olaps

In [46]:
reads = ['ACGGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}


We saw how to adapt dynamic programming to find approximate occurrences of a pattern in a text. Recall that:

-Rows of the dynamic programming matrix are labeled with bases from P and columns with bases from T.

-Elements in the first row are set to 0

-Elements in the first column are set to 0, 1, 2, ..., as for edit distance

-Other elements are set in the same way as elements of a standard edit distance matrix

-The minimal value in the bottom row is the edit distance of the closest match between P and T

In [55]:
#First, download the provided excerpt of human chromosome 1

!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta


--2018-11-07 21:56:06--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.32.81.9, 13.32.81.77, 13.32.81.145, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.32.81.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta.2’

chr1.GRCh38.excerpt 100%[===================>] 791,12K   767KB/s    in 1,0s    

2018-11-07 21:56:07 (767 KB/s) - ‘chr1.GRCh38.excerpt.fasta.2’ saved [810105/810105]



In [61]:
#Second, parse it using the readGenome function we wrote before.

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

genome = readGenome("chr1.GRCh38.excerpt.fasta")

Third, adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below.

#### Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.

In [64]:
def bestApproximateMatchEditDistance(p, t):
    "Returns the edit distance between two strings, p and t"
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    
    # Initialize first row and column of matrix
    for i in range(len(p)+1):   # 0,1,2,3,4,etc en la primera columna
        D[i][0] = i
    # See slide 4 on  0440_approx__editdist3.pdf
    # First row is already initialised to zero
    
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)

    # Respuesta a la tarea. Best Approximate Match Distance is the smallest value of the last row
    return min(D[-1])


In [65]:
p = "GCGTATGC" 
t = "TATTGGCTATACGGTT"
bestApproximateMatchEditDistance(p, t)

2

1. Question 1
What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [66]:
p = "GCTGATCGATCGTACG"
t = genome

bestApproximateMatchEditDistance(p, t)

3

2. Question 2
What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1? (Don't consider reverse complements.)

In [67]:
p = "GATTTACCAGATTGAG"
t = genome

bestApproximateMatchEditDistance(p, t)

2

In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.


Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least k bases long. To make an overlap graph, we could call overlap(a, b, min_length=k) on every possible pair of reads from the dataset. Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read a whose length-6 suffix is GTCCTA. Say GTCCTA does not occur in any other read in the dataset. In other words, the 6-mer GTCCTA occurs at the end of read a and nowhere else. It follows that a's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read a and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of a. This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea. You don't have to do it this way, but this might help you:

- Let every k-mer in the dataset have an associated "Python set object", which starts out empty.
- We use a "Python dictionary" to associate each k-mer with its corresponding set. 

(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer. If our read is GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA. We do this for every read so that, at the end, each set contains all reads containing the corresponding k-mer.

(2) Now, for each read a, we find all overlaps involving a suffix of a. To do this, we take a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding set) and call overlap(a, b, min_length=k) for each.

The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.

In [68]:
#Download and parse the read sequences from the provided Phi-X FASTQ file.
#We'll just use their base sequences, so you can ignore read names and base qualities.
#Also, no two reads in the FASTQ have the same sequence of bases. This makes things simpler.

!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2018-11-07 22:17:56--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.32.81.145, 13.32.81.9, 13.32.81.156, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.32.81.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2,4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2,44M   745KB/s    in 3,4s    

2018-11-07 22:18:00 (745 KB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [70]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [133]:
reads, qualities = readFastq("ERR266411_1.for_asm.fastq")

In [134]:
print(reads[0])

TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC


QUESTION 3 
Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match. Ignore reverse complements.

- Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that. (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.
- Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.
- Hint 3: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called overlap(a, b, min_length=k) on every pair of reads. We also have provided a couple examples you can check against.

Picture the overlap graph corresponding to the overlaps just calculated. How many edges are in the graph? In other words, how many distinct pairs of reads overlap?

In [81]:
#resolucion de la tarea

In [83]:
def getkmers(read, kmer_length):
    """read  GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA."""
    return [read[i:i+kmer_length] for i in range(len(read)+1-kmer_length)]

In [121]:
def overlap_all_pairs(reads, min_length):
    from pprint import pprint
    
    overlap_map = {}
    overlap_graph = {}
    overlap_pairs = []

    #We use a Python dictionary to associate each k-mer with its corresponding set. 
    suffixDict = {}
    for read in reads:
        kmers = getkmers(read, min_length)
        #print(kmers)
        #(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer.
        for kmer in kmers:
            if not kmer in suffixDict.keys():
                #Let every k-mer in the dataset have an associated Python set object, which starts out empty. 
                suffixDict[kmer] = set()
            suffixDict[kmer].add(read)
    #pprint(suffixDict)

    #(2) Now, for each read a, we find all overlaps involving a suffix of a
    for read in reads:
        #we take a's length-k suffix,
        suffix = read[-min_length:]
        #if len(suffix) < min_length:
        #    continue

        #find all reads containing that k-mer (obtained from the corresponding set) ...
        matching_reads = suffixDict[suffix]

        #...and call overlap(a, b, min_length=k) for each.
        for read2 in matching_reads:
            # The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.
            #if read2.find(suffix) >= 0 and read2 != suffix :
            if read2 != read :
                val = overlap(read, read2, min_length) 
                if val > 0:
                    overlap_pairs.append( (read,read2) )
                    overlap_map[ (read, read2) ] = val
                    overlap_graph[read] = read2
                    
    #pprint(overlap_map)
    #pprint(overlap_pairs)
    #pprint(overlap_graph)
    return overlap_pairs, overlap_map, overlap_graph


### Example 1

In [123]:
reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']

In [110]:
def prob(reads):
    lista = []
    for read in reads:
        rp = getkmers(read,3)
        lista.append(rp)
    return lista

In [111]:
prob(reads)

[['ABC', 'BCD', 'CDE', 'DEF', 'EFG'],
 ['EFG', 'FGH', 'GHI', 'HIJ'],
 ['HIJ', 'IJA', 'JAB', 'ABC']]

In [119]:
def dictionary(reads, min_length):
    from pprint import pprint

    suffixDict = {}
    for read in reads:
        kmers = getkmers(read, min_length)
        for kmer in kmers:
            if not kmer in suffixDict.keys():
                suffixDict[kmer] = set()
            suffixDict[kmer].add(read)
    return suffixDict


In [120]:
dictionary(reads, 3)

{'ABC': {'ABCDEFG', 'HIJABC'},
 'BCD': {'ABCDEFG'},
 'CDE': {'ABCDEFG'},
 'DEF': {'ABCDEFG'},
 'EFG': {'ABCDEFG', 'EFGHIJ'},
 'FGH': {'EFGHIJ'},
 'GHI': {'EFGHIJ'},
 'HIJ': {'EFGHIJ', 'HIJABC'},
 'IJA': {'HIJABC'},
 'JAB': {'HIJABC'}}

In [126]:
overlap_all_pairs(reads, 3)

([('ABCDEFG', 'EFGHIJ'), ('EFGHIJ', 'HIJABC'), ('HIJABC', 'ABCDEFG')],
 {('ABCDEFG', 'EFGHIJ'): 3, ('EFGHIJ', 'HIJABC'): 3, ('HIJABC', 'ABCDEFG'): 3},
 {'ABCDEFG': 'EFGHIJ', 'EFGHIJ': 'HIJABC', 'HIJABC': 'ABCDEFG'})

In [128]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
overlap_all_pairs(reads, 5)

([('CGTACG', 'GTACGT'),
  ('CGTACG', 'GTACGA'),
  ('TACGTA', 'ACGTAC'),
  ('GTACGT', 'TACGTA'),
  ('ACGTAC', 'CGTACG'),
  ('GTACGA', 'TACGAT')],
 {('ACGTAC', 'CGTACG'): 5,
  ('CGTACG', 'GTACGA'): 5,
  ('CGTACG', 'GTACGT'): 5,
  ('GTACGA', 'TACGAT'): 5,
  ('GTACGT', 'TACGTA'): 5,
  ('TACGTA', 'ACGTAC'): 5},
 {'ACGTAC': 'CGTACG',
  'CGTACG': 'GTACGA',
  'GTACGA': 'TACGAT',
  'GTACGT': 'TACGTA',
  'TACGTA': 'ACGTAC'})

### Question 3
Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match. Ignore reverse complements.

In [137]:
overlap_pairs, overlap_map, overlap_graph = overlap_all_pairs(reads, 30)

In [142]:
len(overlap_map)

904746

### Question 4

In [143]:
len(overlap_graph)

7161